In [2]:
import sqlite3
import csv

In [3]:
with open('pettigrew.csv', 'r') as fin:
    csvin = csv.reader(fin)
    headers = next(csvin)
    data = [r for r in csvin]

First we want to get the column names out of our data.

In [4]:
headers

['BoxNumber', 'FolderNumber', 'Contents', 'Date']

In [5]:
import sqlite3
conn = sqlite3.connect('create_pettigrew.db')

In [6]:
c = conn.cursor()

Now that we have database created and a cursor to work with, we can craft our create table SQL statment.  The basic syntax for this query is:

``` SQL
CREATE TABLE table_name
             (col1name datatype, col2name datatype);
```

So for our example, we need to know 3 pieces:

1. what we want to call the table in our database.  Remember that you can have multiple tables in a single database, so this table name needs to be something descriptive enough to uniquely identify it but pithy enough that you can type it repeatedly within reason.
2. what the column names will be. This should be pretty reasonable to do because you will likely use them directly from the file that you're reading in.
3. The data types for the columns that you'll be reading in.  Database datatypes deserve their own tutorial, but when in doubt make it text.  Likely your columns will be text, nummber, or some datetype.  Section 2 of this document has more information on the different datatypes:  https://www.sqlite.org/datatype3.html The cool thing is that you can cast things from text within the database, so you can read it in as text and explore from there if you aren't really sure the pros and cons of a certain datatype.  You also need to know your data here, and know what the contents are. Again, if you are unsure, code it as text for the first run through and explore the data from there.   In the case of our example here I will feed you what it should be.

So here are out answers to this question:

1. Let's call it letters
2. Our column names from the headers of the CSV hold what we want to use for the column names.
3. * `BoxNumber`: `integer` because these are all whole numbers
   * `FolderNumber`: `integer` because these are all whole numbers
   * `Contents`: `text` because this is a bunch of text
   * `Date`: `text` because while dates are contained in here, the format is inconsistant and there's extra text in here that we don't want to lose.

So our final SQL command to create this table will be:

``` SQL
CREATE TABLE letters
        (BoxNumber integer, FolderNumber integer, Contents text, Date text)

```

In [8]:
c.execute('''CREATE TABLE letters
        (BoxNumber integer, FolderNumber integer, Contents text, Date text)''')

Now we need to run through our data and add it to the empty table that we just made.  This process should feel something similar to the list accumulator pattern, where we can't add data to a structure that doesn't already exist.

Inserting values commands look like this:

``` SQL
INSERT INTO table_name VALUES (col2val, col2val)
```

So in our case, our first row will need to look like:

``` SQL
INSERT INTO letters VALUES ('1', '1', '[Provenance documents and biographical sources]', 'n.d.')
```

The order that you put the data into the `()` tuple must match the order of the tables that you created in your create table command.

We'll need to play with a few things in here:  

* making the tuple out of our data values
* placing that formatted tuple into our larger select statement.  

Thankfully there's a nice function for handling this for us within the module.

This presumes that we have a list of rows to add, that each item in the row is a separate element in those lists, and that the order in the rows is consistant.  It will also nicely sanitize values, etc. for us.

Unfortunately, the syntax is somewhot cryptic. 

The `c.executemany()` (note the `c` which means this is a method that you are calling on your cursor) command takes two arguments:

1. the first being a string with your insert pattern and `?` characters where you would want it to place your data values.  You'll need to have as many `?` items in there as you have columns.  So instead of trying to do a `.join()` or something, you can use `(?,?,?,?)` if you have four columns.
2. The second argument will be the list of rows that you have.

So putting this together, we get this command to execute:

In [10]:
c.executemany('INSERT INTO letters VALUES (?,?,?,?)', data)

We need to save the changes that we've made to our database now.

In [11]:
conn.commit()

Let's check our work and see what we've done.

In [13]:
results = c.execute("SELECT * FROM letters")
new_data = results.fetchall()

Let's check that our data is the same length as before.  Remember that our previous read in is within the `data` variable, the data coming in from the database was saved in the `new_data` variable.  Their lengths should match.

In [14]:
print(len(data))
print(len(new_data))

601
601


YESSSSSSS.

In [15]:
conn.close()